In [32]:
%gui tk
%matplotlib inline

import numpy as np
import os
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from glob import glob
from skimage import transform
import PIL.Image
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import h5py

In [3]:

IMAGE_HEIGHT = 416
IMAGE_WIDTH = 416
IMAGE_CHANNELS = 3

In [16]:
a = np.array(range(101))

In [18]:
a[0:101:10]


array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [22]:
image = np.zeros((1024,1024,3))

In [20]:
image

array([0., 0., 0., ..., 0., 0., 0.])

In [23]:
image.shape

(1024, 1024, 3)

In [31]:
sl = np.s_[::10]

In [32]:
a[sl]

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [34]:
orig_height, orig_width = image.shape[0], image.shape[1]



In [44]:
a = np.array(range(0,100,5))
b = np.array(range(100,400))
c = np.split(b,a)
c

[array([], dtype=int32),
 array([100, 101, 102, 103, 104]),
 array([105, 106, 107, 108, 109]),
 array([110, 111, 112, 113, 114]),
 array([115, 116, 117, 118, 119]),
 array([120, 121, 122, 123, 124]),
 array([125, 126, 127, 128, 129]),
 array([130, 131, 132, 133, 134]),
 array([135, 136, 137, 138, 139]),
 array([140, 141, 142, 143, 144]),
 array([145, 146, 147, 148, 149]),
 array([150, 151, 152, 153, 154]),
 array([155, 156, 157, 158, 159]),
 array([160, 161, 162, 163, 164]),
 array([165, 166, 167, 168, 169]),
 array([170, 171, 172, 173, 174]),
 array([175, 176, 177, 178, 179]),
 array([180, 181, 182, 183, 184]),
 array([185, 186, 187, 188, 189]),
 array([190, 191, 192, 193, 194]),
 array([195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
        208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
        221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
        234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
        247, 248, 

In [64]:
split_indices_height = np.array(range(IMAGE_HEIGHT,orig_height,IMAGE_HEIGHT))

In [65]:
split_indices_height

array([416, 832])

In [66]:
images_height = np.split(image, split_indices_height, axis = 0)

In [72]:
images_height[2].shape

(192, 1024, 3)

In [76]:
split_indices_height = np.array(range(IMAGE_HEIGHT,orig_height,IMAGE_HEIGHT))        
images_split_height = np.split(image, split_indices_height, axis = 0)
images_split = []
split_indices_width = np.array(range(IMAGE_WIDTH,orig_width,IMAGE_WIDTH))        
for image_split_height in images_split_height:
    images_split += (np.split(image_split_height, split_indices_width, axis = 1))

In [83]:
images_split[5].shape

(416, 192, 3)

In [85]:
bins = split_indices_height
bins

array([416, 832])

In [88]:
x = np.array((20,612, 2, 1000))
x

array([  20,  612,    2, 1000])

In [89]:
y = np.array((323, 1000, 1020, 600))
y

array([ 323, 1000, 1020,  600])

In [96]:
np.digitize(x,bins)

array([0, 1, 0, 2], dtype=int64)

In [101]:
x_indices = np.digitize(x, split_indices_width)
y_indices = np.digitize(y, split_indices_height)
x_indices

array([0, 1, 0, 2], dtype=int64)

In [126]:
y_x_coordinates_matrix = np.empty((len(split_indices_height)+1, len(split_indices_width)+1), dtype = np.object)
y_x_coordinates_matrix

array([[None, None, None],
       [None, None, None],
       [None, None, None]], dtype=object)

In [127]:
y_x_coordinates_matrix[0,0] = []
y_x_coordinates_matrix

array([[list([]), None, None],
       [None, None, None],
       [None, None, None]], dtype=object)

In [151]:
x_y_coordinates = np.array([np.array(x, dtype=np.float32), np.array(y, dtype=np.float32)])
x_y_coordinates

array([[  20.,  612.,    2., 1000.],
       [ 323., 1000., 1020.,  600.]], dtype=float32)

In [159]:
for a,b in zip(x_y_coordinates[0,:], x_y_coordinates[1,:]):
    print(a,b)

20.0 323.0
612.0 1000.0
2.0 1020.0
1000.0 600.0


In [172]:
coordinates_split_height = np.empty(len([1,2,3]), dtype = np.object)
for x,y,y_ind in zip(x_y_coordinates[0,:], x_y_coordinates[1,:],y_indices):
    if coordinates_split_height[y_ind] is None:
        coordinates_split_height[y_ind] = []
    coordinates_split_height[y_ind].append([x,y])
    
coordinates_split_height

array([list([[20.0, 323.0]]), list([[1000.0, 600.0]]),
       list([[612.0, 1000.0], [2.0, 1020.0]])], dtype=object)

In [182]:
coordinates_split = np.empty((len([1,2,3]),len([1,2,3])), dtype = np.object)
for x,y,x_ind,y_ind in zip(x_y_coordinates[0,:], x_y_coordinates[1,:],x_indices,y_indices):
    if coordinates_split[y_ind,x_ind] is None:
        coordinates_split[y_ind,x_ind] = []
    coordinates_split[y_ind,x_ind].append([x,y])
coordinates_split.flatten()


array([[list([[20.0, 323.0]]), None, None],
       [None, None, list([[1000.0, 600.0]])],
       [list([[2.0, 1020.0]]), list([[612.0, 1000.0]]), None]],
      dtype=object)

In [2]:
images_path = "spine_images.hdf5"
image_set = 'validation'
hdf5_file_images = h5py.File(images_path, "r")

data_len = hdf5_file_images["images"].shape[0]

partition = dict(train=np.array(range(int(0.9 * data_len))),
                 validation=np.array(range(int(0.9 * data_len), data_len)))

image_data = hdf5_file_images["images"][partition[image_set],...]
hdf5_file_images.close()

In [15]:
hdf5_file_images.close()

In [24]:
np.expand_dims(image_data,axis = 1)[1].shape

(1, 416, 416, 3)

In [12]:
image = image_data[200]
# plt.figure()
# plt.imshow(image)


(455604,)

In [74]:
file_list = np.load(os.path.normpath('C:\\Users\\smirnovm\\Documents\\Data\\Labeled_Sliding_Window_Spines\\file_list.npz'))['file_list']
data = np.load(file_list[0])
image, boxes = data['image'], data['boxes']
image.shape
boxes.size
list_IDs_temp = [1,2,3,5000,5]
files_to_load = file_list[list_IDs_temp]
images = [np.load(file)['image'] for file in files_to_load]
boxes = [np.load(file)['boxes'] for file in files_to_load]
orig_size = [i.shape for i in images]
orig_size = np.expand_dims(orig_size, axis=0)

# Image preprocessing.
processed_images = [np.stack([i,i,i],2) for i in images]
processed_images = np.array([transform.resize(i,(416, 416,3)) for i in processed_images])

#boxes should be x_center, y_center, box_width, box_height
boxes_processed = []
for box in boxes:
    if box.size>0:
        box[:,0] = box[:,0] + box[:,2]/2
        box[:,1] = box[:,1] + box[:,3]/2
        box = np.concatenate((box,np.ones((box.shape[0],1))),axis = 1)
    else:
        box.reshape(-1,5)
    boxes_processed.append(box)
        
# boxes[1].reshape(-1,5)


c:\users\smirnovm\documents\python scripts\yad2k\venv\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [86]:
# set box parameters as x_center, y_center, box_width, box_height, class
# class is 1 for all boxes
def translate_boxes_to_yolo(boxes):
    yolo_boxes = []
    for box in boxes:
        if box.size > 0:
            box = np.append(box,1)
            box[0] = box[0] + box[2]/2
            box[1] = box[1] + box[3]/2
        else:
            box.reshape(-1,5)
        yolo_boxes.append(box)
    return np.array(yolo_boxes)

In [91]:
yolo_boxes = []
for box in boxes:
    translate_boxes_to_yolo(boxes)
    yolo_boxes.append(boxes)
boxes = yolo_boxes
boxes = [box.reshape((-1, 5)) for box in boxes]

# Get box parameters as x_center, y_center, box_width, box_height, class.
# boxes_xy = [0.5 * (box[:, 3:5] + box[:, 1:3]) for box in boxes]
# boxes_wh = [box[:, 3:5] - box[:, 1:3] for box in boxes]
boxes_xy = [box[:,:2] / orig_size for box in boxes]
boxes_wh = [box[:,2:] / orig_size for box in boxes]
boxes = [np.concatenate((boxes_xy[i], boxes_wh[i], box[:, 0:1]), axis=1) for i, box in enumerate(boxes)]


AttributeError: 'list' object has no attribute 'reshape'

In [94]:
yolo_boxes

[[array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64),
  array([[ 87.51393443, 135.24672131,  26.66721311,  26.66721311],
         [122.75983607, 139.34508197,  26.66721311,  26.66721311]]),
  array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64)],
 [array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64),
  array([[ 87.51393443, 135.24672131,  26.66721311,  26.66721311],
         [122.75983607, 139.34508197,  26.66721311,  26.66721311]]),
  array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64)],
 [array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64),
  array([[ 87.51393443, 135.24672131,  26.66721311,  26.66721311],
         [122.75983607, 139.34508197,  26.66721311,  26.66721311]]),
  array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64)],
 [array([], shape=(0, 4), dtype=float64),
  array([], shape=(0, 4), dtype=float64),
 